In [4]:
import numpy as np 
import pandas as pd 
import cv2
import os
from PIL import Image
from IPython.display import HTML

In [5]:
DATADIR = "C:/Users/sanky/OneDrive/Documents/aima-python-master/PetImages"

CATEGORIES = ["Dog", "Cat"]

for category in CATEGORIES:  # do dogs and cats
    path = os.path.join(DATADIR,category)  # create path to dogs and cats
    for img in os.listdir(path):  # iterate over each image per dogs and cats
        img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
       

        break  # we just want one for now so break
    break  #...and one more!

In [6]:
print(img_array)

[[117 117 119 ... 133 132 132]
 [118 117 119 ... 135 134 134]
 [119 118 120 ... 137 136 136]
 ...
 [ 79  74  73 ...  80  76  73]
 [ 78  72  69 ...  72  73  74]
 [ 74  71  70 ...  75  73  71]]


In [7]:
print(img_array.shape)

(375, 500)


In [8]:
IMG_SIZE = 80

new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))

In [9]:
new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))

In [10]:
training_data = []

def create_training_data():
    for category in CATEGORIES:  # do dogs and cats

        path = os.path.join(DATADIR,category)  # create path to dogs and cats
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=dog 1=cat

        for img in os.listdir(path):  # iterate over each image per dogs and cats
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass
            #except OSError as e:
            #    print("OSErrroBad img most likely", e, os.path.join(path,img))
            #except Exception as e:
            #    print("general exception", e, os.path.join(path,img))

create_training_data()

print(len(training_data))

24946


In [11]:
import random

random.shuffle(training_data)
for sample in training_data[:10]:
    print(sample[1])

0
1
1
0
0
0
0
1
1
0


In [12]:
X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)

print(X[0].reshape(-1, IMG_SIZE, IMG_SIZE, 1))

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

[[[[ 31]
   [ 21]
   [ 20]
   ...
   [167]
   [ 57]
   [ 79]]

  [[  7]
   [ 17]
   [ 19]
   ...
   [ 53]
   [ 56]
   [ 72]]

  [[  6]
   [ 17]
   [112]
   ...
   [ 53]
   [ 53]
   [ 69]]

  ...

  [[  8]
   [ 17]
   [ 24]
   ...
   [134]
   [123]
   [139]]

  [[  6]
   [ 21]
   [ 21]
   ...
   [ 91]
   [100]
   [135]]

  [[  4]
   [ 15]
   [ 14]
   ...
   [ 68]
   [127]
   [128]]]]


In [13]:
import pickle

pickle_out = open("X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [14]:
pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

In [15]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import numpy as np

import pickle

pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)


X=np.array(X/255.0)
y=np.array(y)

model = Sequential()

model.add(Conv2D(256, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X, y, batch_size=32, epochs=10,validation_split=0.3)

Epoch 1/10
546/546 [==============================] - 531s 973ms/step - loss: 0.6758 - accuracy: 0.6019 - val_loss: 0.6337 - val_accuracy: 0.6379
Epoch 2/10
546/546 [==============================] - 520s 953ms/step - loss: 0.6005 - accuracy: 0.6822 - val_loss: 0.5943 - val_accuracy: 0.6860
Epoch 3/10
546/546 [==============================] - 517s 947ms/step - loss: 0.5432 - accuracy: 0.7291 - val_loss: 0.5604 - val_accuracy: 0.7129
Epoch 4/10
546/546 [==============================] - 520s 953ms/step - loss: 0.5054 - accuracy: 0.7568 - val_loss: 0.5423 - val_accuracy: 0.7281
Epoch 5/10
546/546 [==============================] - 523s 957ms/step - loss: 0.4684 - accuracy: 0.7823 - val_loss: 0.5099 - val_accuracy: 0.7594
Epoch 6/10
546/546 [==============================] - 516s 945ms/step - loss: 0.4389 - accuracy: 0.7995 - val_loss: 0.5140 - val_accuracy: 0.7596
Epoch 7/10
546/546 [==============================] - 506s 927ms/step - loss: 0.4159 - accuracy: 0.8068 - val_loss: 0.5036 -

In [104]:
img_array = cv2.imread("C:/Users/sanky/OneDrive/Documents/aima-python-master/PetImages/Cat/4.jpg" ,cv2.IMREAD_GRAYSCALE)

In [105]:
new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))

In [106]:
X_test = np.array(new_array).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

In [107]:
X_test=np.array(X_test/255.0)

In [108]:
model.predict(X_test)

array([[0.7904881]], dtype=float32)

In [109]:
model.predict_classes(X_test)

array([[1]])